##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pruning para a inferência no dispositivo com XNNPACK

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_for_on_device_inference"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/model_optimization/guide/pruning/pruning_for_on_device_inference.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Bem-vindo ao guia sobre o pruning de pesos do Keras para melhorar a latência da inferência no dispositivo via [XNNPACK](https://github.com/google/XNNPACK).

Este guia apresenta o uso da API `tfmot.sparsity.keras.PruningPolicy` recém-lançada e demonstra como ela pode ser usada para acelerar principalmente modelos convolucionais em CPUs modernas usando a [inferência esparsa do XNNPACK](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/delegates/xnnpack/README.md#sparse-inference).

O guia abrange as seguintes etapas do processo de criação do modelo:

- Compilar e treinar o modelo
- Ajustar o modelo com o pruning
- Converter para o TFLite
- Fazer o benchmarking no dispositivo

O guia não cobre as práticas recomendadas para os ajustes com o pruning. Para informações mais detalhadas sobre esse tópico, confira nosso [guia completo](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md).

## Configuração

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization

In [ ]:
import tempfile

import tensorflow as tf
import numpy as np

from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

%load_ext tensorboard

## Compile e treine o modelo denso

Compilamos e treinamos uma CNN de referência simples para a tarefa de classificação no dataset [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html).

In [ ]:
# Load CIFAR10 dataset.
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train[:90%]', 'train[90%:]', 'test'],
    as_supervised=True,
    with_info=True,
)

# Normalize the input image so that each pixel value is between 0 and 1.
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.image.convert_image_dtype(image, tf.float32), label

# Load the data in batches of 128 images.
batch_size = 128
def prepare_dataset(ds, buffer_size=None):
  ds = ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.cache()
  if buffer_size:
    ds = ds.shuffle(buffer_size)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  return ds

ds_train = prepare_dataset(ds_train,
                           buffer_size=ds_info.splits['train'].num_examples)
ds_val = prepare_dataset(ds_val)
ds_test = prepare_dataset(ds_test)

# Build the dense baseline model.
dense_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(32, 32, 3)),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.Conv2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=(3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=16, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.DepthwiseConv2D(
        kernel_size=(3, 3), strides=(2, 2), padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=32, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Compile and train the dense model for 10 epochs.
dense_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

dense_model.fit(
  ds_train,
  epochs=10,
  validation_data=ds_val)

# Evaluate the dense model.
_, dense_model_accuracy = dense_model.evaluate(ds_test, verbose=0)

## Criar o modelo esparso

Usando as instruções do [guia completo](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md), aplicamos a função `tfmot.sparsity.keras.prune_low_magnitude` com parâmetros direcionados à aceleração no dispositivo pelo pruning, ou seja, a política `tfmot.sparsity.keras.PruneForLatencyOnXNNPack`.

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after after 5 epochs.
end_epoch = 5

num_iterations_per_epoch = len(ds_train)
end_step =  num_iterations_per_epoch * end_epoch

# Define parameters for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.25,
                                                               final_sparsity=0.75,
                                                               begin_step=0,
                                                               end_step=end_step),
      'pruning_policy': tfmot.sparsity.keras.PruneForLatencyOnXNNPack()
}

# Try to apply pruning wrapper with pruning policy parameter.
try:
  model_for_pruning = prune_low_magnitude(dense_model, **pruning_params)
except ValueError as e:
  print(e)

A chamada `prune_low_magnitude` resulta em `ValueError` com a mensagem `Could not find a GlobalAveragePooling2D layer with keepdims = True in all output branches`. Essa mensagem indica que o modelo não é compatível com o pruning pela política `tfmot.sparsity.keras.PruneForLatencyOnXNNPack` e, especificamente, a camada `GlobalAveragePooling2D` exige o parâmetro `keepdims = True`. Vamos corrigir isso e aplicar a função `prune_low_magnitude`.

In [ ]:
fixed_dense_model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(32, 32, 3)),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.Conv2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=(3, 3), padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=16, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.ZeroPadding2D(padding=1),
    keras.layers.DepthwiseConv2D(
        kernel_size=(3, 3), strides=(2, 2), padding='valid'),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=32, kernel_size=(1, 1)),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(keepdims=True),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Use the pretrained model for pruning instead of training from scratch.
fixed_dense_model.set_weights(dense_model.get_weights())

# Try to reapply pruning wrapper.
model_for_pruning = prune_low_magnitude(fixed_dense_model, **pruning_params)

A invocação de `prune_low_magnitude` foi concluída sem nenhum erro, o que significa que o modelo é totalmente compatível com a política `tfmot.sparsity.keras.PruneForLatencyOnXNNPack` e pode ser acelerado usando a [inferência esparsa de XNNPACK](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/delegates/xnnpack/README.md#sparse-inference).

### Ajuste o modelo esparso

Seguindo o [exemplo de pruning](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras.md), ajustamos o modelo esparso usando os pesos do modelo denso. Começamos a ajustar o modelo com uma esparsidade de 25% (25% dos pesos são definidos como zero) e terminamos com uma esparsidade de 75%.

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

model_for_pruning.fit(
  ds_train,
  epochs=15,
  validation_data=ds_val,
  callbacks=callbacks)

# Evaluate the dense model.
_, pruned_model_accuracy = model_for_pruning.evaluate(ds_test, verbose=0)

print('Dense model test accuracy:', dense_model_accuracy)
print('Pruned model test accuracy:', pruned_model_accuracy)

Os logs mostram a progressão da esparsidade em cada camada.

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir={logdir}

Depois de fazer os ajustes com o pruning, a exatidão de teste mostra uma melhoria modesta (de 43% a 44%) em relação ao modelo denso. Vamos comparar a latência no dispositivo usando o [benchmark do TFLite](https://www.tensorflow.org/lite/performance/measurement).

## Conversão e benchmarking do modelo

Para converter o modelo podado para o TFLite, precisamos substituir os wrappers `PruneLowMagnitude` com camadas originais através da função `strip_pruning`. Além disso, como os pesos do modelo podado (`model_for_pruning`) são, em grande parte, zeros, podemos aplicar uma otimização `tf.lite.Optimize.EXPERIMENTAL_SPARSITY` para armazenar com eficiência o modelo do TFLite gerado. Essa flag de otimização não é necessária para o modelo denso.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(dense_model)
dense_tflite_model = converter.convert()

_, dense_tflite_file = tempfile.mkstemp('.tflite')
with open(dense_tflite_file, 'wb') as f:
  f.write(dense_tflite_model)

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.EXPERIMENTAL_SPARSITY]
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')
with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

Seguindo as instruções da [Ferramenta de Benchmarking de Modelo do TFLite](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/tools/benchmark), compilamos a ferramenta, fazemos o upload dela para o dispositivo Android com os modelos do TFLite densos e podados e fazemos o benchmarking dos dois modelos no dispositivo.

In [ ]:
! adb shell /data/local/tmp/benchmark_model \
    --graph=/data/local/tmp/dense_model.tflite \
    --use_xnnpack=true \
    --num_runs=100 \
    --num_threads=1

In [ ]:
! adb shell /data/local/tmp/benchmark_model \
    --graph=/data/local/tmp/pruned_model.tflite \
    --use_xnnpack=true \
    --num_runs=100 \
    --num_threads=1

Os benchmarks no Pixel 4 resultam em um tempo de inferência médio de *17us* para o modelo denso e *12us* para o modelo podado. Os benchmarks no dispositivo demonstram uma melhoria clara de **5us** ou **30%** na latência, mesmo para modelos tão pequenos. Em nossa experiência, modelos grandes baseados em [MobileNetV3](https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v3) ou [EfficientNet-lite](https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet/lite) mostram melhorias semelhantes no desempenho. Os speed-up variam com base na contribuição relativa das convoluções 1x1 para o modelo em geral.


## Conclusão

Neste tutorial, mostramos como criar modelos esparsos para desempenho mais rápido no dispositivo usando a nova funcionalidade lançada pela API MOT do TF e pelo XNNPack. Esses modelos esparsos são menores e mais rápidos do que suas contrapartes densas, além de reterem ou até mesmo superarem a qualidade.

Recomendamos que você teste essa nova capacidade que pode ser especialmente importante para implantar seus modelos no dispositivo.